In [15]:
# Instalar RDKit
!pip install rdkit-pypi

In [17]:
# Importamos las librerías necesarias
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from google.colab import files

In [18]:
# Cargar el archivo .csv desde tu ordenador
uploaded = files.upload()

Saving All_SMILES.csv to All_SMILES.csv


In [19]:
# Ignorar las líneas que no tienen el número esperado de campos
df = pd.read_csv('All_SMILES.csv', on_bad_lines='skip')

# Mostrar las primeras filas para verificar
df.head()

,NatUVDB,ApprovedDrugs,LANaPDB
0,COC1=CC(=CC(=C1O)OC)CCC2=CC(=C(C=C2)O)OC,CC1=NC=C(COP(O)(O)=O)C(C=O)=C1O,COc1ccc(-c2oc3cc(O)c(OC)c(O)c3c(=O)c2OC)cc1
1,COC1=C(C(=C2C(=C1)C=CC3=C2C=CC(=C3OC)O)OC)O,C[S+](CC[C@H](N)C([O-])=O)C[C@H]1O[C@H]([C@H](...,C=C1C(=O)O[C@H]2[C@H]1CC[C@]1(C)[C@@H]2C(C)=CC...
2,COC1=CC(=O)C2=C(C1=O)CCC3=C2C=CC(=C3)O,N[C@@H](CC1=CC=CC=C1)C(O)=O,COC1=CC(=O)[C@@H]2O[C@]2(C)[C@H]1O
3,COC1=C(C=C(C=C1)CCC2=CC(=CC(=C2)OC)O)O,[H][C@]12CS[C@@H](CCCCC(O)=O)[C@@]1([H])NC(=O)N2,COC1=CC(=O)C(O)=C(C)C1=O
4,COC1=C(C(=C2C(=C1)CCC3=C2C=CC(=C3)O)OC)O,C[N+](C)(C)CCO,COC1=C(O)C(=O)C(O)=C(C)C1=O


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys, DataStructs
import numpy as np

In [ ]:
# Función para calcular las MACCS keys de una molécula
def mol_to_maccs(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return MACCSkeys.GenMACCSKeys(mol)
    else:
        return None

# Función para calcular la diversidad química basada en la similitud Tanimoto con MACCS keys
def calculate_diversity(smiles_list):
    fps = [mol_to_maccs(smiles) for smiles in smiles_list if mol_to_maccs(smiles) is not None]

    if len(fps) < 2:
        return 0  # No se puede calcular diversidad con menos de 2 moléculas

    # Matriz de similitudes
    similarity_matrix = np.zeros((len(fps), len(fps)))

    for i in range(len(fps)):
        for j in range(i+1, len(fps)):
            similarity = DataStructs.FingerprintSimilarity(fps[i], fps[j])
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity

    # Diversidad como 1 - promedio de la similitud
    avg_similarity = np.mean(similarity_matrix[similarity_matrix != 0])
    diversity = 1 - avg_similarity

    return diversity

# Calcular diversidad para cada columna usando MACCS keys
for column in ["NatUVDB", "ApprovedDrugs", "LANaPDB"]:
    smiles_list = df[column].dropna().tolist()
    diversity = calculate_diversity(smiles_list)
    print(f"Diversidad química en {column} (MACCS): {diversity}")


Diversidad química en NatUVDB (MACCS): 0.6023251590451963


[15:28:49] SMILES Parse Error: syntax error while parsing: OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1
[15:28:49] SMILES Parse Error: Failed parsing SMILES 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1' for input: 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1'


Diversidad química en ApprovedDrugs (MACCS): 0.6878274137404348
Diversidad química en LANaPDB (MACCS): 0.5287576750970554


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

In [ ]:
# Función para calcular el fingerprint ECFP4 de una molécula
def mol_to_ecfp4(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)  # ECFP4 es Morgan con radio 2
    else:
        return None

# Función para calcular la diversidad química basada en la similitud Tanimoto con ECFP4
def calculate_diversity(smiles_list):
    fps = [mol_to_ecfp4(smiles) for smiles in smiles_list if mol_to_ecfp4(smiles) is not None]

    if len(fps) < 2:
        return 0  # No se puede calcular diversidad con menos de 2 moléculas

    # Matriz de similitudes
    similarity_matrix = np.zeros((len(fps), len(fps)))

    for i in range(len(fps)):
        for j in range(i+1, len(fps)):
            similarity = DataStructs.FingerprintSimilarity(fps[i], fps[j])
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity

    # Diversidad como 1 - promedio de la similitud
    avg_similarity = np.mean(similarity_matrix[similarity_matrix != 0])
    diversity = 1 - avg_similarity

    return diversity

# Calcular diversidad para cada columna usando ECFP4
for column in ["NatUVDB", "ApprovedDrugs", "LANaPDB"]:
    smiles_list = df[column].dropna().tolist()
    diversity = calculate_diversity(smiles_list)
    print(f"Diversidad química en {column} (ECFP4): {diversity}")


Diversidad química en NatUVDB (ECFP4): 0.8811547799376097


[15:39:06] SMILES Parse Error: syntax error while parsing: OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1
[15:39:06] SMILES Parse Error: Failed parsing SMILES 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1' for input: 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1'


Diversidad química en ApprovedDrugs (ECFP4): 0.8969243437862459
Diversidad química en LANaPDB (ECFP4): 0.8676169403483784


In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from scipy.spatial.distance import pdist, squareform

In [ ]:
# Función para obtener propiedades tipo "drug-like" de una molécula
def calculate_drug_like_properties(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mw = Descriptors.MolWt(mol)  # Peso molecular
        logp = Descriptors.MolLogP(mol)  # LogP
        h_donors = Lipinski.NumHDonors(mol)  # Número de grupos donantes de H
        h_acceptors = Lipinski.NumHAcceptors(mol)  # Número de grupos aceptores de H
        num_rings = Descriptors.RingCount(mol)  # Número de anillos
        num_double_bonds = sum(1 for bond in mol.GetBonds() if bond.GetBondTypeAsDouble() == 2.0)  # Contar enlaces dobles

        return [mw, logp, h_donors, h_acceptors, num_rings, num_double_bonds]
    else:
        return None

# Función para calcular la diversidad basada en propiedades tipo "drug-like"
def calculate_drug_like_diversity(smiles_list):
    properties = [calculate_drug_like_properties(smiles) for smiles in smiles_list]
    properties = np.array([prop for prop in properties if prop is not None])

    if properties.shape[0] < 2:
        return 0  # No se puede calcular diversidad con menos de 2 moléculas

    # Calcular la matriz de distancias
    distance_matrix = squareform(pdist(properties, metric='euclidean'))

    # Diversidad como inverso de la suma de distancias
    diversity = np.mean(distance_matrix[distance_matrix != 0])

    return diversity

# Calcular diversidad para cada columna usando propiedades tipo "drug-like"
for column in ["NatUVDB", "ApprovedDrugs", "LANaPDB"]:
    smiles_list = df[column].dropna().tolist()
    diversity = calculate_drug_like_diversity(smiles_list)
    print(f"Diversidad química en {column} (propiedades tipo drug-like): {diversity:.4f}")


Diversidad química en NatUVDB (propiedades tipo drug-like): 233.6450


[16:06:40] SMILES Parse Error: syntax error while parsing: OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1
[16:06:40] SMILES Parse Error: Failed parsing SMILES 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1' for input: 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1'


Diversidad química en ApprovedDrugs (propiedades tipo drug-like): 227.5785
Diversidad química en LANaPDB (propiedades tipo drug-like): 135.5192


In [23]:
import pandas as pd
import numpy as np
from scipy.integrate import trapz
from rdkit import Chem
from rdkit.Chem import DataStructs
from rdkit.Chem import AllChem

In [29]:
# Función para calcular el CSR (sustituye esto con tu cálculo real)
def calcular_csr(smiles):
    x = np.linspace(0, 10, 100)  # Tiempo u otra variable
    y = np.abs(np.sin(x) + np.random.normal(0, 0.1, len(x)))  # Simulando una curva CSR
    return x, y

# Función para calcular la similitud entre dos SMILES utilizando RDKit
def calcular_similitud(smiles1, smiles2):
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)
    if mol1 is not None and mol2 is not None:
        fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, radius=2)
        fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, radius=2)
        return DataStructs.FingerprintSimilarity(fp1, fp2)
    else:
        return None  # Retorna None si los SMILES son inválidos

# Crear un DataFrame para almacenar los resultados de CSR y similitudes
results = []

# Calcular el CSR y la similitud
for index, row in df.iterrows():
    for col in ["NatUVDB", "ApprovedDrugs", "LANaPDB"]:
        smiles = row[col]
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:  # Solo procesar SMILES válidos
            x, y = calcular_csr(smiles)  # Calcular CSR
            auc = trapz(y, x)  # Calcular área bajo la curva
            results.append({'SMILES': smiles, 'AUC': auc, 'Scaffold': col})

# Convertir resultados a un DataFrame
results_df = pd.DataFrame(results)

# Calcular similitudes, ignorando SMILES inválidos
for i, row1 in results_df.iterrows():
    similitudes = []
    for j, row2 in results_df.iterrows():
        if i != j:
            sim = calcular_similitud(row1['SMILES'], row2['SMILES'])
            if sim is not None:  # Solo añadir similitudes válidas
                similitudes.append(sim)
    results_df.at[i, 'Similarities'] = similitudes

# Calcular el área bajo la curva usando similitudes como eje x
# Aquí asumimos que tomaremos la media de las similitudes para cada grupo
for scaffold in results_df['Scaffold'].unique():
    scaffold_data = results_df[results_df['Scaffold'] == scaffold]
    if not scaffold_data.empty:  # Verificar que hay datos
        similarities = np.concatenate(scaffold_data['Similarities'].values.tolist())
        auc_values = scaffold_data['AUC'].values

        if len(similarities) > 0 and len(auc_values) > 0:  # Verificar que no estén vacíos
            # Calcular AUC de similitudes y CSR
            auc_simi = trapz(auc_values, similarities)

            # Graficar
            plt.plot(similarities, auc_values, label=scaffold)
            plt.fill_between(similarities, auc_values, alpha=0.3)

            print(f'Área bajo la curva (AUC) para {scaffold}: {auc_simi:.2f}')

# Personalizar la gráfica
plt.title('Área Bajo la Curva de CSR vs Similitud Molecular')
plt.xlabel('Similitud Molecular')
plt.ylabel('Valores de CSR (AUC)')
plt.legend()
plt.grid()
plt.show()


<ipython-input-29-9336f08c9a36>:28: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.trapezoid' and will be removed in SciPy 1.14.0
  auc = trapz(y, x)  # Calcular área bajo la curva


TypeError: No registered converter was able to produce a C++ rvalue of type std::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> > from this Python object of type float